In [26]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mysql.connector

### LECTURA DATOS

In [27]:
mysql_config = {
    'host': 'localhost',
    'user': 'root',
    'password': 'MagicNL12',
    'database': 'cancer_mama'
}

conn_mysql = mysql.connector.connect(**mysql_config)
cursor_mysql = conn_mysql.cursor()

query = "SELECT * FROM tumor;"
df_sql = pd.read_sql_query(query, conn_mysql)

C:\Users\34699\AppData\Local\Temp\ipykernel_11076\1996938415.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn_mysql)


In [34]:
df_csv1 = pd.read_csv('./Datos/patients_batch1.csv',sep=',')
df_csv2 = pd.read_csv('./Datos/patients_batch2.csv',sep=',')
df_excel = pd.read_excel('./Datos/gynecological.xlsx')
df_json = pd.read_json('./Datos/histochemistry.json')